In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, sample_each, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [6]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(2000, 1035)

# run

## initialize log

In [7]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [8]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 0 0 

## run

In [9]:
while epoch < config.n_epochs:
    time_start = time.time()
    sample_each(train_docs, topic_root, train=True)
    time_log = float(time.time() - time_start)
    
    sample_each(dev_docs, topic_root, train=False)
    sample_each(test_docs, topic_root, train=False)
    
#     ppl_train = get_perplexity(train_docs, topic_root)
    ppl_train=0
    ppl_dev = get_perplexity(dev_docs, topic_root)
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, ppl_dev, ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN:      VALID:       TEST: SPEC:             HIER:  \
        Time  Ep Ct    PPL         PPL         PPL     1     2     3 CHILD   
0   3.621286   0  0      0  492.246171  489.819360  0.06  0.26  0.49  0.80   
1   3.582841   1  0      0  489.128421  486.079049  0.06  0.28  0.50  0.81   
2   3.586289   2  0      0  487.643450  482.551692  0.06  0.29  0.51  0.82   
3   3.678328   3  0      0  484.991543  481.537293  0.05  0.31  0.51  0.83   
4   4.022076   4  0      0  481.375700  478.748393  0.06  0.32  0.50  0.84   
5   3.675666   5  0      0  483.159743  478.748393  0.05  0.43  0.51  0.92   
6   3.910927   6  0      0  480.981582  476.711541  0.05  0.44  0.51  0.92   
7   3.553432   7  0      0  481.600271  476.711541  0.05  0.44  0.51  0.92   
8   3.812012   8  0      0  480.374335  477.568072  0.06  0.43  0.51  0.92   
9   3.752115   9  0      0  477.880824  475.953773  0.06  0.44  0.51  0.92   
10  4.364723  10  0      0  478.781906  475.953773  0.06  0.44  0.51  0.92   
11  3.513193  11  0      0  475.953050  474.035447  0.06  0.44  0.51  0.91   
12  3.376728  12  0      0  476.904489  474.035447  0.06  0.43  0.51  0.91   
13  4.780311  13  0      0  476.865157  474.035447  0.05  0.44  0.52  0.91   
14  3.444591  14  0      0  476.190816  474.035447  0.05  0.44  0.51  0.92   
15  3.471226  15  0      0  474.634735  472.081017  0.06  0.44  0.51  0.92   
16  3.371433  16  0      0  475.605125  472.081017  0.05  0.44  0.51  0.92   
17  3.579194  17  0      0  478.644827  472.081017  0.05  0.43  0.51  0.92   
18  3.710565  18  0      0  477.871270  472.081017  0.05  0.44  0.51  0.92   
19  3.572526  19  0      0  476.313750  472.081017  0.05  0.45  0.51  0.92   
20  3.730164  20  0      0  479.063392  472.081017  0.05  0.44  0.51  0.92   
21  3.298654  21  0      0  477.537488  472.081017  0.05  0.44  0.50  0.92   
22  4.798873  22  0      0  474.888077  472.081017  0.05  0.45  0.51  0.92   
23  4.172323  23  0      0  476.256037  472.081017  0.05  0.45  0.51  0.91   
24  4.098961  24  0      0  475.298202  472.081017  0.05  0.45  0.51  0.92   
25  3.787207  25  0      0  475.897205  472.081017  0.06  0.45  0.51  0.92   
26  4.016066  26  0      0  476.183239  472.081017  0.05  0.45  0.51  0.92   
27  3.347763  27  0      0  476.000155  472.081017  0.05  0.45  0.51  0.91   
28  5.425397  28  0      0  476.492290  472.081017  0.05  0.45  0.51  0.92   
29  3.361432  29  0      0  475.078504  472.081017  0.05  0.45  0.51  0.91   
..       ...  .. ..    ...         ...         ...   ...   ...   ...   ...   
70  3.354534  70  0      0  471.616524  463.297476  0.06  0.45  0.51  0.90   
71  4.417244  71  0      0  468.632614  463.297476  0.06  0.45  0.51  0.89   
72  3.269434  72  0      0  470.886592  463.297476  0.06  0.45  0.51  0.89   
73  6.060117  73  0      0  469.890040  463.297476  0.06  0.45  0.51  0.89   
74  3.447204  74  0      0  472.681095  463.297476  0.06  0.44  0.51  0.90   
75  5.006190  75  0      0  472.985986  463.297476  0.06  0.44  0.51  0.90   
76  4.006336  76  0      0  471.221000  463.297476  0.06  0.45  0.51  0.89   
77  5.002877  77  0      0  471.239843  463.297476  0.06  0.45  0.51  0.89   
78  3.396447  78  0      0  472.647119  463.297476  0.06  0.45  0.51  0.90   
79  4.479630  79  0      0  470.351957  463.297476  0.06  0.45  0.51  0.90   
80  3.404990  80  0      0  467.796848  463.297476  0.06  0.45  0.50  0.89   
81  4.579179  81  0      0  468.127391  463.297476  0.06  0.44  0.51  0.89   
82  3.547029  82  0      0  465.908231  462.421226  0.06  0.45  0.51  0.88   
83  3.865643  83  0      0  463.431033  460.378830  0.06  0.45  0.51  0.88   
84  3.472339  84  0      0  462.558147  458.128383  0.06  0.46  0.51  0.87   
85  3.533751  85  0      0  457.425208  453.624740  0.06  0.45  0.51  0.86   
86  3.479939  86  0      0  449.403228  447.146324  0.06  0.45  0.51  0.83   
87  3.531523  87  0      0  442.808698  439.497409  0.06  0.46  0.51  0.80   
88  3.570767  88  0      0  440.235752  43

 0 2000 22429.0 ! nice sleeve bought ; love price & quality perfect
   0-1 949 6641.0 carry strap pockets pocket shoulder handle compartment hold space straps
     0-1-1 949 722.0 lunch past zip papers straps carrying pouch usb shoulder exterior
   0-2 1051 5516.0 cover color keyboard air bottom mac hard scratches pro retina
     0-2-1 1051 746.0 cute buying blue offers - edges speck loose colors green
